-----------------
-----------------
###  Making Bases!

Below, I'll convert my old pairing.html javascript code from the
[Lie Algebra / coalgebra pairing calculator](https://drive.google.com/file/d/1j3pHzz9CqklB8YfgB96CbUD1RmiZlN_f/view)
to python. I'll also write a new algorithm to generate the star basis for symbols.

This section will mostly target Lie algebra - coalgebra pairings.  The next section will look at finitely presented groups and bases for configuration braiding invariants.

Note: this code requires the [coLie.py](https://cocalc.com/share/public_paths/4a54e77fae1a6040ef22aefcab5b0ca127a38420) library of coLie objects.

In [26]:
import math


def genLS_old(word):
    """Use Duval's algorithm (1988) to generate all Lyndon(-Shirshov) words using given alphabet and multiplicities"""
    
    # create alphabet = list of (letter,count) pairs
    alphabet = sorted(list(set(word)))
    count    = [word.count(letter) for letter in alphabet]

    
    if alphabet == []:
        return
    
    N = len(word)
   
    LSWord = [alphabet[0]]
    
    # apply algorithm from Duval (1988)
    
    while LSWord != []:
        if len(LSWord) == len(word) and all([LSWord.count(l)==n for l,n in zip(alphabet,count)]):
            yield(''.join(LSWord))
            
        LSWord = LSWord * math.ceil(N/len(LSWord))
                
        n = N-1
        
        while n >= 0 and LSWord[n] == alphabet[-1]:
            n -= 1
            
        LSWord = LSWord[:n+1]
        
        if n >= 0:
            LSWord[n] = alphabet[alphabet.index(LSWord[n])+1]
            
    return



# An alternate algorithm is given by Cattell et al (2000) 
#   https://www.cis.uoguelph.ca/~sawada/papers/un.pdf
def genLS_all(word, t=1, p=1, N=None, K=None, root=True, LSWord=None):
    if root:
        N = len(word)
        if N < 2:
            return
        
        LSWord = [0] * (N + 1)    # Cattell's algorithm uses 1-indexing... ugh
        word   = sorted(list(set(word)))  # later iterations need alphabet
        K      = len(word)
        
        
    if t > N:
        if p == N:
            yield(''.join(word[num] for num in LSWord[1:]))
    else:
        LSWord[t] = LSWord[t-p]
        genLS_all(word, t+1, p, N, K, False, LSWord)
        
        for j in range(LSWord[t-p]+1, K):
            LSWord[t] = j
            genLS_all(word, t+1, t, N, K, False, LSWord)


            
# It is possible to modify this algorithm to generate only words with specific multiplicies of letters
#  this is modification of code at http://combos.org/necklace
class LListElement:
    def __init__(self,value=0,index=0,n=None,p=None):
        self.value , self.index = value , index
        self._prev , self._next = p , n
        
    def __bool__(self):
        return self.value != 0

class ValuedLList:
    def __init__(self,count):
        if len(count) < 1:
            return
         
        self._nodes  = [LListElement(count[0])]
        for n in range(1,len(count)):
            self._nodes.append(LListElement(count[n],n,self._nodes[n-1]))
            self._nodes[n-1]._prev = self._nodes[n]
            
        self.head = self._nodes[-1]
    
    def mult(self,n):
        return self._nodes[n].value
    
    def decrement(self,n):
        self._nodes[n].value -= 1
        
        if not self._nodes[n]:
            if self.head == self._nodes[n]:
                self.head = self._nodes[n]._next
                
            if self._nodes[n]._next:
                self._nodes[n]._next._prev = self._nodes[n]._prev
            if self._nodes[n]._prev:
                self._nodes[n]._prev._next = self._nodes[n]._next
        
    def increment(self,n):
        if not self._nodes[n]:
            if self._nodes[n]._next:
                self._nodes[n]._next._prev = self._nodes[n]
            if self._nodes[n]._prev:
                self._nodes[n]._prev._next = self._nodes[n]
            else:
                self.head = self._nodes[n]
                
        self._nodes[n].value += 1
        
    def nextval(self,n):
        if self._nodes[n]._next:
            return self._nodes[n]._next.index
        return -1
                
def genLS(word, t=2, p=1, s=2, N=None, K=None, root=True, LSWord=None, count=None, tmp=None):
    if root:
        N = len(word)
        if N < 2:
            return
        
        tmp    = sorted(list(set(word)))
        count  = ValuedLList([word.count(letter) for letter in tmp])
        word   = tmp              # later iterations need alphabet
        K      = len(word)
        LSWord = [K] * (N + 1)    # Cattell's algorithm uses 1-indexing... ugh
        tmp    = [0] * (N + 1)
        
        LSWord[1] = 0             # Cattell's algorithm uses 1-indexing... ugh
        count.decrement(0)
        
    if count.mult(-1) == N-t+1:
        if count.mult(-1) == tmp[t-p] and N == p:
            yield(''.join(word[num] for num in LSWord[1:]))
        elif count.mult(-1) > tmp[t-p]:
            yield(''.join(word[num] for num in LSWord[1:]))
            
    elif count.mult(0) != N-t+1:
        j = count.head.index
        ss = s
        while j >= LSWord[t-p]:
            tmp[s]    = t-s
            LSWord[t] = j
            
            count.decrement(j)
            
            if j != K:
                ss = t+1
            if j == LSWord[t-p]:
                genLS(word,t+1,p,ss,N,K,False,LSWord,count,tmp)
            else:
                genLS(word,t+1,t,ss,N,K,False,LSWord,count,tmp)
                
            count.increment(j)
            
            j = count.nextval(j)
            
        LSWord[t] = K-1

#####################
#  Lyndon words are minimal in their cyclic ordering class
#   usually we use lexicographic ordering
#   an alternate ordering, proposed by Chibrikov is the deg-lex ordering
#     a < aa < aaa < aaaa
#
def genDL():
    passs

#//////////////////////////////////////////////////////////////////////////////// 
#// toDLWord
#//   converts LS word to DL word 
#//
#function toDLWord( word ) {
#	return toDL( word.split(''), word.charAt(0) );
#}
#function toDL( word, wordMin ) {
#	if (word.length <=1 )
#			return word[0];
#
#	var i, j;
#	var newWord = [];
#	var newMin = "999999999999999999";
#
#	for (i=0; (i < word.length) && (word[i] != wordMin); i++) ;
#
#	j = i;
#
#	while (i < word.length+j)  {
#
#		var subWord = word[i];
#
#		for (i++ ; (i < word.length) && (word[i] == wordMin); i++)  {
#			subWord = subWord+word[i];
#		}
#		
#		if (i != word.length)
#			subWord = subWord+word[i];
#		else 
#			i--;
#
#		for (i++ ; (i < word.length + j) && (word[i] != wordMin); i++)
#			subWord = subWord+word[(i % word.length)];
#
#		newWord.push( subWord );
#		if (parseInt(subWord.replace(/\D/g,'')) < parseInt(newMin.replace(/\D/g,''))) 
#			newMin = subWord;
#	}
#
#	return toDL( newWord, newMin );
#}


In [27]:
genLS("aaabbbd")


abadabb
ababbad
ababadb
abababd
aadbbab
aadbabb
aadabbb
aabdbab
aabdabb
aabbdab
aabbbad
aabbadb
aabbabd
aabadbb
aababdb
aababbd
aaadbbb
aaabdbb
aaabbdb
aaabbbd


In [5]:
#######################################################################
# Code below makes Lie bases from LS words using different rules      #
#######################################################################

from coLie import *

###########################################################
# code below is direct translation of my old javascript code to Python without optimization or cleaning
#
# This makes the classical bracketing on an LS word - recursively defined as
#    B(w) = [ B(a) , B(b) ]  where w = ab and b is a maximal LS subword
#
#  ... this probably isn't the best algorithm...
def bracketStd(word):
    if len(word) == 1:
        return LieTree(word)
    
    end = len(word)-1
    newprefix , myprefix = 1 , 0
    newcut = end
    
    repeat = True
    
    for j in reversed(range(1,len(word)-2)):
        if repeat:
            if word[j] == word[end]:
                end -= 1
            else:
                repeat = False
                end = len(word) - 1
                
        if word[j] > word[newcut]:
            myprefix = 0
        else:
            if word[j] == word[newcut]:
                myprefix += 1
                
                if not repeat:
                    if myprefix == newprefix: # compare new cutpoint to old cutpoint
                        i = 1
                        while word[i+j] == word[newcut]:
                            i += 1
                        if word[i+j] < word[newcut]:
                            newcut = j
                            repeat = True
                    elif myprefix > newprefix: # found longer prefix -- new cutpoint
                        newprefix = myprefix
                        newcut = j
                        repeat = True
            else:                              # found new low -- new cutpoint
                newprefix = 1
                myprefix = 1
                newcut = j
                repeat = True
    
    bracket = LieTree()
    bracket.bracket = [ bracketStd(word[:newcut]) , bracketStd(word[newcut:]) ]
    
    return bracket  


# see also algorithm by Sawada and Ruskey (2002) .. generating words and brackets at same time
#  https://webhome.cs.uvic.ca/~ruskey/Publications/LieBasis/LieBasis.pdf
#
# algorithm is implemented in C at http://combos.org/necklace




###########################################################
# code below is translated from my javascript code
#
# Note: this is a basis by [WaSh15] and pairs nicely with the star basis for symbols
def bracketLeft(word):
    if len(word) <= 1:
        return LieTree(word)
    
    i , j = 0 , 1
    
    while j < len(word)-1 and word[i] <= word[j]:
        if word[i] == word[j]:
            i += 1
        else:
            i = 0
        j += 1
    
    bracket = LieTree()
    bracket.bracket = [ bracketLeft(word[:j-i]) , bracketLeft(word[j-i:]) ]
    
    return bracket



###########################################################
# code below is translated from my old javascript code
#
# I'm pretty sure this is a basis, but I don't remember if I/anyone ever proved it....
def bracketRight(word):
    return bracketRG(list(word))

def bracketRG(word):
    if len(word) == 1:
        return word[0] if isinstance(word[0],LieTree) else LieTree(word[0])

    newWord = []
    
    i = 1
    while i < len(word):
        subbracket = bracketRG([word[i-1]])
        
        while i < len(word) and str(word[i]) != str(word[0]):
            tmpbracket = LieTree()
            tmpbracket.bracket = [ bracketRG([subbracket]) , bracketRG([word[i]]) ]
            subbracket = tmpbracket
            i += 1
        
        newWord.append(subbracket)
        i += 1
        
    return bracketRG(newWord)



###########################################################        
# "configuration pairing".. this is a "left-greedy word sensitive bracketing"
# This is the reverse of Chibrikov ... it has a particularly nice pairing with LS words 
#
# this is a direct translation of my javascript code to python
def bracketConfig(word):
    return LieTree(bracketCfg(list(word)))


def bracketCfg(word):
    if len(word) == 1:
        return word[0]
    
    newWord = []
    
    start , i = 0 , 1 
    
    while i < len(word):
        bracket = ['[', word[start] ]
        
        while word[i] == word[0]:
            bracket.extend([ ',[' , word[i] ])
            i += 1
        bracket.extend(["," , word[i]])
        bracket.extend(["]"] * (i-start))
                
        i += 1
        while i < len(word) and word[i] != word[0]:
            bracket.insert(0, '[')
            bracket.extend([',' , word[i] , ']'])
            i += 1
            
        newWord.append(''.join(bracket))
                
        start = i
        i += 1
        
    return bracketCfg(newWord)
    


###########################################################
# "right normed" basis of Chibrikov:
#   https://core.ac.uk/download/pdf/82357333.pdf
#
# this is a direct translation of my javascript code to python
def bracketChib(word):
    return LieTree(bracketChibrikov(list(word)))

def bracketChibrikov(word):
    if len(word) == 1:
        return word[0]
    
    newWord = []
    
    end , i = len(word)-1 , len(word)-2     

    while i>=0:
        bracket = [word[end], ']']
        
        while word[i] != word[0]:
            bracket.insert(0,']')
            bracket.insert(0,word[i])
            i -= 1

        bracket.insert(0,word[i])
        
        while end > i:
            bracket.insert(0,'[')
            end -= 1
        
        i -= 1
        while i >= 0 and word[i] == word[0]:
            bracket.insert(0,word[i])
            bracket.insert(0,'[')
            bracket.append(']')
            i -= 1
            
        newWord.insert(0,''.join(bracket))
        
        end = i
        i -= 1
        
    return bracketChibrikov(newWord)


In [6]:
word = next(genLS("aaabbb"))
print(EilWord(word) * bracketLeft(word))

print(EilTree("(((((a)a)a)b)b)b") * bracketLeft(word))

print(bracketChibrikov(["a","a","a","b","b","b"]))


print(" Standard:")

gen = genLS("aaabbb")
print(bracketLeft(next(gen)))
print(bracketLeft(next(gen)))
print(bracketLeft(next(gen)))

print("\n Left-Greedy:")

gen = genLS("aaabbb")
print(bracketLeft(next(gen)))
print(bracketLeft(next(gen)))
print(bracketLeft(next(gen)))

print("\n Right-Greedy:")

gen = genLS("aaabbb")
print(bracketRight(next(gen)))
print(bracketRight(next(gen)))
print(bracketRight(next(gen)))


print("\n Reverse Chibrikov")

gen = genLS("aaabbb")
print(bracketConfig(next(gen)))
print(bracketConfig(next(gen)))
print(bracketConfig(next(gen)))


print("\n  Chibrikov")

gen = genLS("aaabbb")
print(bracketChib(next(gen)))
print(bracketChib(next(gen)))
print(bracketChib(next(gen)))

1
1
[a[a[[[ab]b]b]]]
 Standard:
[[[a,[a,[a,b]]],b],b]
[[[a,[a,b]],[a,b]],b]
[[[a,[a,b]],b],[a,b]]

 Left-Greedy:
[[[a,[a,[a,b]]],b],b]
[[[a,[a,b]],[a,b]],b]
[[[a,[a,b]],b],[a,b]]

 Right-Greedy:
[a,[a,[[[a,b],b],b]]]
[[a,[a,b]],[[a,b],b]]
[[a,[[a,b],b]],[a,b]]

 Reverse Chibrikov
[[[a,[a,[a,b]]],b],b]
[[a,[a,b]],[[a,b],b]]
[[[a,[a,b]],b],[a,b]]

  Chibrikov
[a,[a,[[[a,b],b],b]]]
[[a,[a,b]],[[a,b],b]]
[[a,[[a,b],b]],[a,b]]
